In [1]:
# Ensure src/ is in sys.path so 'tools' can be imported
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(TOOLS_PATH)


if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/netflix_talk2data/src


In [4]:
# src/graph/build_graph.py

from langgraph.graph import StateGraph, START
from agents.supervisor_agent import supervisor_node
from agents.enhancer_agent import enhancer_node
from agents.qdrant_search_agent import quadrant_search_node

def build_graph():
    """
    Constructs and compiles the LangGraph workflow:

      START
        ↓
    supervisor_node
        ├──> enhancer_node ───┐
        │       ↑             │
        │       └─────────────┘
        └──> quadrant_search_node -> END

    Returns:
        A compiled graph ready to invoke or stream.
    """
    builder = StateGraph()

    # Register each node
    builder.add_node("supervisor", supervisor_node)
    builder.add_node("enhancer", enhancer_node)
    builder.add_node("qdrant_search", quadrant_search_node)

    # Define where execution begins
    builder.set_entry_point("supervisor")

    # Static edges for validation (dynamic goto inside supervisor handles actual flow)
    builder.add_edge(START, "supervisor")
    builder.add_edge("supervisor", "enhancer")
    builder.add_edge("enhancer", "supervisor")
    builder.add_edge("supervisor", "qdrant_search")

    # Mark the end node
    builder.set_finish_point("qdrant_search")

    # Compile into an executable graph
    return builder.compile()

In [7]:
# 3. Display it in‐notebook
from IPython.display import Image, display
import tempfile
import subprocess

# Ensure mermaid-cli is installed
import sys
if subprocess.call(["which", "mmdc"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) != 0:
	print("Installing mermaid-cli globally using npm...")
	# Install mermaid-cli if not present
	subprocess.check_call(["npm", "install", "-g", "@mermaid-js/mermaid-cli"])

# Example Mermaid diagram as a string
mermaid_code = """
graph TD
	START --> supervisor
	supervisor --> enhancer
	enhancer --> supervisor
	supervisor --> qdrant_search
	qdrant_search --> END
"""

# Write Mermaid code to a temporary file
with tempfile.NamedTemporaryFile("w", suffix=".mmd", delete=False) as mmd_file:
	mmd_file.write(mermaid_code)
	mmd_path = mmd_file.name

# Output PNG path
png_path = mmd_path.replace(".mmd", ".png")

# Generate PNG using mermaid CLI (requires mermaid-cli installed: npm install -g @mermaid-js/mermaid-cli)
subprocess.run(["mmdc", "-i", mmd_path, "-o", png_path], check=True)

# Read the PNG as bytes
with open(png_path, "rb") as f:
	mermaid_png = f.read()

display(Image(data=mermaid_png))

Installing mermaid-cli globally using npm...


FileNotFoundError: [Errno 2] No such file or directory: 'npm'